# Audio Processing

> Audio extraction, conversion, and processing functions

In [ ]:
#| default_exp audio

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
import subprocess
from typing import Union
from cjm_ffmpeg_utils.core import get_audio_codec
from cjm_ffmpeg_utils.media_info import get_media_duration, get_audio_info_ffmpeg
from cjm_ffmpeg_utils.execution import run_ffmpeg_with_progress

## Audio Extraction

In [ ]:
#| export
def extract_audio(input_path: Path, # Path to the input video file
                  output_path: Path, # Path for the output audio file
                  audio_format: str = 'mp3', # Output audio format (mp3, wav, flac, aac, etc.)
                  verbose: bool = False # If True, shows detailed ffmpeg output
                  ):
    """
    Extract audio from a video file using ffmpeg.
    """
    # Check if input file exists
    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {input_path}")
    
    # Get duration of input file
    total_duration = get_media_duration(input_path)
    
    # Ensure output path has correct extension
    if not str(output_path).endswith(f'.{audio_format}'):
        output_path = output_path.with_suffix(f'.{audio_format}')
    
    # FFmpeg command to extract audio
    cmd = [
        'ffmpeg',
        '-i', str(input_path),           # Input video file
        '-vn',                           # No video (audio only)
        '-acodec', 'copy' if audio_format == 'original' else get_audio_codec(audio_format),  # Audio codec
        '-progress', 'pipe:2',           # Send progress to stderr
        '-y',                            # Overwrite output file if it exists
        str(output_path)                 # Output audio file
    ]
    
    # Add format-specific options
    if audio_format == 'mp3':
        cmd.extend(['-ab', '192k'])      # Set bitrate for MP3
    elif audio_format == 'wav':
        cmd.extend(['-ar', '44100'])     # Set sample rate for WAV
    elif audio_format == 'flac':
        cmd.extend(['-compression_level', '5'])  # FLAC compression
    
    # Run ffmpeg with progress bar
    run_ffmpeg_with_progress(
        cmd=cmd,
        total_duration=total_duration,
        description="Extracting audio",
        verbose=verbose
    )
    
    print(f"\nSuccessfully extracted audio to {audio_format.upper()} format.")

## Audio Downsampling

In [ ]:
#| export
def downsample_audio(input_path: Path, # Path to the input audio file
                     output_path: Path, # Path for the output file
                     sample_rate: Union[int, str] = "16k", # Audio bitrate
                     channels: Union[int, str] = "1", # Audio channels
                     overwrite: bool = True, # Overwrite existing output file
                     verbose: bool = False # If True, shows detailed ffmpeg output
                    ):
    """
    Downsample an audio file to 16kbps and single channel using ffmpeg.
    """
    # Get duration of input file
    total_duration = get_media_duration(input_path)
    
    # FFmpeg command to downsample to 16kbps mono
    cmd = [
        'ffmpeg',
        '-i', str(input_path),           # Input file
        '-ac', f'{channels}',            # Set audio channels to 1 (mono)
        '-ab', f'{sample_rate}',         # Set audio bitrate to 16kbps
        '-progress', 'pipe:2',           # Send progress to stderr
        str(output_path)                 # Output file
    ]
    
    if overwrite:
        cmd.append('-y') # Overwrite output file if it exists
    
    # Run ffmpeg with progress bar
    run_ffmpeg_with_progress(
        cmd=cmd,
        total_duration=total_duration,
        description="Downsampling",
        verbose=verbose
    )
    
    print(f"\nSuccessfully downsampled audio.")

## Audio Format Conversion

In [ ]:
#| export
def convert_to_mp3(input_path: Path, # Path to the input audio file
                   output_path: Path, # Path where the MP3 file will be saved
                   bitrate: str = "128k", # Audio bitrate for the output MP3 file
                   verbose: bool = False # Whether to display verbose output during conversion
                   ) -> Path: # Path to the converted MP3 file
    """
    Convert an audio file to MP3 format.
    """
    # Get audio duration (note: fixed typo from get_audio_duration to get_media_duration)
    total_duration = get_media_duration(input_path)
    
    cmd = [
        'ffmpeg',
        '-i', str(input_path),
        '-codec:a', 'libmp3lame',
        '-b:a', bitrate,
        '-progress', 'pipe:2',
        '-y',
        str(output_path)
    ]
    
    run_ffmpeg_with_progress(
        cmd=cmd,
        total_duration=total_duration,
        description="Converting to MP3",
        verbose=verbose
    )
    
    print(f"\nSuccessfully converted to MP3.")
    
    return output_path

## Audio Segmentation

In [ ]:
#| export
def extract_audio_segment(input_path: Path, # Path to the input audio file
                          output_path: Path, # Path where the extracted segment will be saved
                          start_time: str, # Start time in format "HH:MM:SS" or seconds
                          duration: str, # Duration in format "HH:MM:SS" or seconds
                          verbose: bool = False, # Whether to show verbose output
                          pbar: bool = False # Whether to show a progress bar
                        ):
    """
    Extract a segment from an audio file.
    """
    # For segments, we can calculate the expected duration
    try:
        # Try to parse duration as seconds first
        segment_duration = float(duration)
    except ValueError:
        # If that fails, assume it's in HH:MM:SS format
        time_parts = duration.split(':')
        if len(time_parts) == 3:
            hours = float(time_parts[0])
            minutes = float(time_parts[1])
            seconds = float(time_parts[2])
            segment_duration = hours * 3600 + minutes * 60 + seconds
        else:
            segment_duration = None
    
    cmd = [
        'ffmpeg',
        '-i', str(input_path),
        '-ss', start_time,
        '-t', duration,
        '-progress', 'pipe:2',
        '-y',
        str(output_path)
    ]

    if pbar:
        run_ffmpeg_with_progress(
            cmd=cmd,
            total_duration=segment_duration,
            description="Extracting segment",
            verbose=verbose
        )
    else:
        result = subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            text=True
        )

        if result.returncode == 0:
            print(f"Successfully extracted audio segment.")
        else:
            raise subprocess.CalledProcessError(result.returncode, cmd)

## Testing

In [ ]:
#|eval: false
from cjm_ffmpeg_utils.core import FFMPEG_AVAILABLE

if FFMPEG_AVAILABLE:
    # Test with a video file
    test_video = Path("alpha_meditation_video.mp4")
    if test_video.exists():
        # Extract audio
        audio_output = test_video.with_suffix(".mp3")
        extract_audio(test_video, audio_output, audio_format="mp3")
        
        # Downsample audio
        downsampled = audio_output.with_stem(f"{audio_output.stem}_downsampled")
        downsample_audio(audio_output, downsampled)
        
        # Extract a segment
        segment = audio_output.with_stem(f"{audio_output.stem}_segment")
        extract_audio_segment(audio_output, segment, "0", "30", pbar=True)

Extracting audio:   0%|          | 0.0/5.0s [00:00<?]


Successfully extracted audio to MP3 format.


Downsampling:   0%|          | 0.0/5.0s [00:00<?]


Successfully downsampled audio.


Extracting segment:   0%|          | 0.0/30.0s [00:00<?]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()